In [12]:
import os
from typing import Optional, Union

from smolagents import CodeAgent, ToolCallingAgent, HfApiModel, tool, DuckDuckGoSearchTool

In [13]:
@tool
def calculate(operation: str, numbers: object) -> float:
    """Performs basic mathematical operations on a list of numbers.

    Args:
        operation: One of 'sum', 'average', 'multiply', 'min', 'max'
        numbers: List of numbers to operate on

    Returns:
        float: Result of the operation
    """
    # Note - when used with ToolCallingAgent, `numbers` are a dictionary.
    # TODO: How to handle this or primarily give the input as a list of numbers.
    if operation == "sum":
        return sum(numbers)
    elif operation == "average":
        return sum(numbers) / len(numbers)
    elif operation == "multiply":
        result = 1
        for n in numbers:
            result *= n
        return result
    elif operation == "min":
        return min(numbers)
    elif operation == "max":
        return max(numbers)
    else:
        raise ValueError(f"Unknown operation: {operation}")


# Create agent with custom tool
math_agent = CodeAgent(tools=[calculate], model=HfApiModel())

# Test the agent
response = math_agent.run("What is the average of 10, 15, 20, 25, and 30?")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the average of 10, 15, 20, 25, and 30?                                                                  │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

RESPONSE ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop_sequence', index=0, message=ChatCompletionOutputMessage(role='assistant', content='Thought: I will use the \'calculate\' tool with the \'average\' operation to compute the average of the given numbers: 10, 15, 20, 25, and 30.\nCode:\n```py\naverage_result = calculate(operation="average", numbers=[10, 15, 20, 25, 30])\nfinal_answer(average_result)\n```<end_code>', tool_calls=None), logprobs=None)], created=1737653423, id='', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint='3.0.1-sha-bb9095a', usage=ChatCompletionOutputUsage(completion_tokens=91, prompt_tokens=2109, total_tokens=2200))


─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  average_result = calculate(operation="average", numbers=[10, 15, 20, 25, 30])                                    
  final_answer(average_result)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 20.0

[Step 0: Duration 0.18 seconds| Input tokens: 2,109 | Output tokens: 91]

20.0


In [14]:
search_tool = DuckDuckGoSearchTool()

# The reason to not use ToolCallingAgent for `calculate` is mentioned above
search_agent = CodeAgent(
    model=HfApiModel(),
    tools=[search_tool, calculate]
)

search_agent.run("Which is easier - to find out the capital of France or to calculate the average of 10, 20, and 30?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Which is easier - to find out the capital of France or to calculate the average of 10, 20, and 30?              │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

RESPONSE ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop_sequence', index=0, message=ChatCompletionOutputMessage(role='assistant', content='Thought: I need to determine the ease of finding out the capital of France and calculating the average of the numbers 10, 20, and 30. I will use the `web_search` tool for the first task and the `calculate` tool for the second task. I will then compare the results to determine which is easier.\n\nCode:\n```py\n# Finding out the capital of France\ncapital_france = web_search(query="capital of France")\nprint("Capital of France:", capital_france)\n\n# Calculating the average of 10, 20, and 30\naverage_numbers = calculate(operation="average", numbers=[10, 20, 30])\nprint("Average of 10, 20, and 30:", average_numbers)\n```<end_code>', tool_calls=None), logprobs=None)], created=1737653817, id='', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint='3.0.1-sha-bb9095a', usage=ChatCompletionOutputUsage(completion_token

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  # Finding out the capital of France                                                                              
  capital_france = web_search(query="capital of France")                                                           
  print("Capital of France:", capital_france)                                                                      
                                                                                                                   
  # Calculating the average of 10, 20, and 30                                                                      
  average_numbers = calculate(operation="average", numbers=[10, 20, 30])                                           
  print("Average of 10, 20, and 30:", average_numbers)                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Capital of France: ## Search Results

[Paris - Wikipedia](https://en.wikipedia.org/wiki/Paris)
Paris (French pronunciation: ⓘ) is the capital and largest city of France.With an estimated population of 2,102,650
residents in January 2023 [2] in an area of more than 105 km 2 (41 sq mi), [5] Paris is the fourth-most populous 
city in the European Union, the ninth-most populous city in Europe and the 30th most densely populated city in the 
world in 2022. [6]

[Paris | Definition, Map, Population, Facts, & History | Britannica](https://www.britannica.com/place/Paris)
Paris, city and capital of France, located along the Seine River, in the north-central part of the country. Paris 
is one of the world's most important and attractive cities, famed for its gastronomy, haute couture, painting, 
literature, and intellectual community. Learn more about Paris in this article.

[Paris - Simple English Wikipedia, the free encyclopedia](https://simple.wikipedia.org/wiki/Paris)
Paris is the capital city and the largest city of France, with a population of about 2.15 million. It has a rich 
history, many art museums, historical buildings, and a famous landmark, the Eiffel Tower.

[What is the Capital of France? - 
WorldAtlas](https://www.worldatlas.com/articles/what-is-the-capital-of-france.html)
Learn about the history, geography, economy, tourism, and administration of Paris, the capital city of France and 
the country's largest city. Find out why Paris is called the City of Light and the City of Love.

[List of capitals of France - Wikipedia](https://en.wikipedia.org/wiki/List_of_capitals_of_France)
The capital of France has been Paris since its liberation in 1944. [1] Chronology. Tournai (before 486), 
current-day Belgium; Soissons (486-936) Laon (936-987) Paris (987-1419), the residence of the Kings of France, 
although they were consecrated at Reims.

[France | History, Maps, Flag, Population, Cities, Capital, & Facts ...](https://www.britannica.com/place/France)
France, a country of northwestern Europe, is historically and culturally among the most important countries in the 
Western world. It has also played a highly significant role in international affairs for centuries. Its capital is 
Paris, one of the most important cultural and commercial centers in the world.

[Paris: Facts & Related Content - Encyclopedia Britannica](https://www.britannica.com/facts/Paris)
Learn about Paris, the capital of France, and its history, culture, landmarks, and people. Find out why Paris is 
called "the City of Light" and when it was founded, liberated, and attacked.

[Paris, France - New World Encyclopedia](https://www.newworldencyclopedia.org/entry/Paris,_France)
Learn about the history, geography, culture, and administration of Paris, the capital city of France and one of the
world's leading global cities. Find out how Paris got its name, what landmarks and monuments it has, and how it 
became a center of education and ideas.

[What is the Capital of France? - GeeksforGeeks](https://www.geeksforgeeks.org/what-is-the-capital-of-france/)
Learn why Paris is the capital of France and what makes it a global center for art, fashion, culture, and history. 
Discover its iconic landmarks, such as the Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and the Arc de 
Triomphe.

[Paris - capital city of France - The World Countries](https://theworldcountries.com/place/paris/)
Learn about the history, geography, culture, and attractions of Paris, the city and capital of France. Explore its 
landmarks, museums, parks, and rivers, and discover its role as a global centre of arts, education, and commerce.
Average of 10, 20, and 30: 20.0

Out: None

...

RESPONSE ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop_sequence', index=0, message=ChatCompletionOutputMessage(role='assistant', content='Thought: Based on the observations, finding out the capital of France is straightforward and the search results clearly state that Paris is the capital. Calculating the average of 10, 20, and 30 is also straightforward and the result is 20.0. However, since the calculation is more directly mathematical and does not require interpreting human language, I would argue that calculating the average is easier in terms of computational effort. But finding out the capital of France involves retrieving and understanding natural language information, which may be considered easier by some because it is a common piece of knowledge that can be quickly verified without formal computation.\n\nHowever, if we strictly consider the computational effort, calculating the average is simpler. I will provide the answer based on computation

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Calculating the average of 10, 20, and 30 is easier in terms of computational effort.")            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Calculating the average of 10, 20, and 30 is easier in terms of computational effort.

[Step 1: Duration 4.38 seconds| Input tokens: 5,563 | Output tokens: 363]

'Calculating the average of 10, 20, and 30 is easier in terms of computational effort.'